In [1]:
#Import important libraries
import numpy as np
import pandas as pd

In [2]:
# STAGE_1_DIR = '/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/'
# SEASON_DIR = '/kaggle/input/mncaa-additional-data-2022/'
teams = pd.read_csv('MTeams.csv')
teams.head()

# advance_stats = pd.read_csv('advance_stats.csv', encoding='cp1252')
# advance_stats.info()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2020
1,1102,Air Force,1985,2020
2,1103,Akron,1985,2020
3,1104,Alabama,1985,2020
4,1105,Alabama A&M,2000,2020


In [3]:
def compute_average_rank(teamID, season, rankings_df=None):
    if rankings_df is None:
        rankings_df = pd.read_csv('MMasseyOrdinals.csv')
    ranks = rankings_df[(rankings_df.TeamID == teamID) & (rankings_df.Season == season)]
    return ranks.OrdinalRank.mean()

# Example: 
# print("Alabama's average rank for 2020 is", compute_average_rank(1104, 2020))

In [4]:
def clean_season_data(dataframe, season, teamIDs=None, rankings=None):
    ''' Cleans the data by removing rows, filling in NA values, etc. '''
    print(f'Processing data for {season} season')

    # convert team names to team id
    if teamIDs is None:
        teamIDs = pd.read_csv('MTeamSpellings.csv', encoding='cp1252')
    dfIDs = []
    ranks = []

    for i, sch in enumerate(dataframe.School):
        team = teamIDs[teamIDs.TeamNameSpelling.str.lower() == sch]
        if len(team) == 0:
            # some entries were not in the data (see table below)
            if sch == 'purdue-fort wayne':
                schid = 1236
            elif sch == 'cal state long beach':
                schid = np.nan
            elif sch == 'kansas city':
                schid = 1282
            elif sch == 'st. thomas (mn)':
                schid = 1472
            else:
                schid = int(input(sch))
        else:
            schid = int(team.TeamID.iloc[0])
        dfIDs.append(schid)
        ranks.append(compute_average_rank(schid, int(season),
                                          rankings_df=rankings))
    dataframe['ID'] = dfIDs
    
    # add season column
    season_data = [int(season)] * len(dataframe)
    dataframe['Season'] = season_data
    
    # add rank column
    dataframe['Rank'] = ranks
    
    return dataframe

In [5]:
def process_season_data(data):
    ''' Reads in the data, cleans, and returns a dataframe '''
    teamIDs = pd.read_csv('MTeamSpellings.csv', encoding='cp1252')
    rankings_df = pd.read_csv('MMasseyOrdinals.csv')
    dataframes = []
    for d in data:
        df = pd.read_csv(d, header=1)
        df = clean_season_data(df, '20' + d.rstrip('.csv')[-2:],
                               teamIDs=teamIDs, rankings=rankings_df)
        dataframes.append(df)
    return pd.concat(dataframes)

In [6]:
season_stats = pd.read_csv('basic_stats.csv', encoding='cp1252')

team_season_stats = clean_season_data(season_stats, 2014)
# team_season_stats.head() 

Processing data for 2014 season


Abilene Christian 2014


KeyboardInterrupt: Interrupted by user

In [7]:
team_season_stats.describe()

NameError: name 'team_season_stats' is not defined